# Phi-4 Mini ONNX பரலல் செயல்பாட்டு அழைப்பு பயிற்சி

இந்த நோட்புக் Phi-4 Mini ஐ ONNX Runtime GenAI உடன் பரலல் செயல்பாட்டு அழைப்புக்காக எப்படி பயன்படுத்துவது என்பதை விளக்குகிறது. செயல்பாட்டு அழைப்பு நம்பிக்கைமிக்க முறையில் பயனர் வேண்டுதல்களின் அடிப்படையில் வெளிப்புற கருவிகள் மற்றும் APIகளை அழைக்க முடியத்தக்க வகையாக மாடலை அனுமதிக்கிறது.

## மேலோட்டம்

இந்த பயிற்சியில், நீங்கள் கீழ்க்காணும் விஷயங்களை கற்றுக்கொள்ளப் போகிறீர்கள்:
- Phi-4 Mini ஐ ONNX Runtime GenAI உடன் அமைக்க
- விமானங்களையும் ஹோட்டல்களையும் முன்பதிவு செய்வதற்கான செயல்பாட்டு ஸ்கீமாக்களை வரையறுக்க
- கட்டமைக்கப்பட்ட வெளியீட்டுக்காக Lark இலக்கணத்துடன் வழிநடத்தப்பட்ட உருவாக்கத்தை பயன்படுத்தவும்
- சிக்கலான பயண முன்பதிவு சூழ்நிலைகளுக்காக பரலல் செயல்பாட்டு அழைப்புகளை இயக்கவும்

## முன்-தேவைகள்

இந்த நோட்புக் இயக்குவதற்கு முன், கீழ்க்காணும் பொருட்கள் உங்களிடம் உள்ளதை உறுதிசெய்க:
- Phi-4 Mini ONNX மாடலை பதிவிறக்கம் செய்திருக்க வேண்டும்
- `onnxruntime-genai` தொகுதி நிறுவப்பட்டிருக்க வேண்டும்
- செயல்பாட்டு அழைப்புச் சம்பந்தப்பட்ட அடிப்படை புரிதல்


## படி 1: தேவையான நூலகங்களை இறக்குமதி செய்க

முதலில், நமது செயல்பாடுகளை அழைக்கும் செயலாக்கத்திற்காக தேவையான நூலகங்களை நாம் இறக்குமதி செய்வோம்.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## படி 2: மாடல் அமைப்பு மற்றும் உள்ளமைவு

இப்போது நாம் Phi-4 Mini ONNX மாடலை அமைக்கப்போகிறோம். பாதையை உங்கள் உண்மையான மாடல் அடைவாக மாற்றியிடுவதை உறுதி செய்யவும்.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## படி 3: உருவாக்க அளவுருக்களை அமைக்கவும்

மாடலின் வெளியீட்டு நடத்தை கட்டுப்படுத்த உருவாக்க அளவுருக்களை அமைக்கவும். இந்த அமைப்புகள் செயல்பாட்டு அழைப்புகளுக்கு தீர்மானமான மற்றும் கவனமிக்க பதில்களை உறுதி செய்கின்றன.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## படி 4: கிடைக்கும் செயல்பாடுகளை வரையறுக்கவும்

இங்கு எங்கள் AI உதவியாளர் அழைக்கக்கூடிய செயல்பாடுகளை வரையறுக்கிறோம். இந்த உதாரணத்தில், இரண்டு பயண சம்பந்தமான செயல்பாடுகள் உள்ளன:
1. **booking_flight_tickets**: விமான நிலையங்களுக்கிடையிலான விமானங்களை முன்பதிவு செய்வதற்காக
2. **booking_hotels**: ஹோட்டல் வசதிகளை முன்பதிவு செய்வதற்காக

செயல்பாட்டு வரையறைகள் OpenAI-இன் function calling schema வடிவத்தை பின்பற்றுகின்றன.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## படி 5: இலக்கண உருவாக்க உதவி செயல்பாடுகள்

இந்த உதவி செயல்பாடுகள் எங்கள் செயல்பாடு வரையறைகளை Lark இலக்கண வடிவத்திற்கு மாற்றுகிறது, இது வழிகாட்டப்பட்ட உருவாக்கத்திற்கு ONNX Runtime GenAI பயன்படுத்துகிறது. இதனால் மாடல் சரியான JSON வடிவில் செல்லுபடியாகும் செயல்பாடு அழைப்புகளை வெளியிடுவதை உறுதி செய்கிறது.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## படி 6: இலக்கண உருவாக்கத்தை சோதிக்கவும்

எங்கள் கருவி வரையறைகளை சரியான வடிவமாக மாற்ற அவை எப்படி செயல்படுகின்றன என்பதை காண எமது இலக்கண உருவாக்க செயல்பாடுகளை சோதிப்போம்.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## படி 7: சிஸ்டம் பிராம்ப்ட் மற்றும் ஜெனரேட்டர் தயாரிக்கவும்

இப்போது நாங்கள் மாடலுக்கு கிடைக்கும் கருவிகள் பற்றி கூறும் சிஸ்டம் பிராம்ப்டை உருவாக்கி, வழிகாட்டப்பட்ட உருவாக்க அளவுருக்கள் கொண்டு ஜெனரேட்டரை அமைப்போம்.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## படி 8: வழிகாட்டப்பட்ட உருவாக்கத்துடன் ஜெனரேட்டரை துவக்குதல்

இப்போது நாங்கள் எங்கள் கட்டமைக்கப்பட்ட அளவுருக்களுடன் ஜெனரேட்டரை உருவாக்கி வழிகாட்டப்பட்ட உருவாக்கத்திற்காக Lark இலக்கணத்தைப் பயன்படுத்துவோம்.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## படி 9: ஒரே நேரத்தில் பல செயல்பாடுகளை அழைக்க சோதிக்கவும்

இப்போது பல செயல்களை அழைக்க வேண்டிய ஒரு சிக்கலான பயண முன்பதிவு சூழ்நிலையுடன் நமது அமைப்பை சோதிப்போம்.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### வடிவமைக்கப்பட்ட பயனர் கோரிக்கையை பார்க்கவும்


### செயல்பாட்டு அழைப்புகள் உருவாக்கவும்

மாதிரி இப்போது எங்கள் பயனர் கோரிக்கையின் அடிப்படையில் கட்டமைக்கப்பட்ட செயல்பாட்டு அழைப்புகளை உருவாக்கும். வழிநடத்தப்பட்ட உருவாக்கத்தின் மூலம், வெளியீடு நேரடியாக இயங்கக்கூடிய செல்லுபடியாகும் JSON வடிவில் இருக்கும்.

**எதிர்பார்க்கப்படும் வெளியீடு**: மாதிரி கீழ்காணும் செயல்பாட்டு அழைப்புகளை உருவாக்க வேண்டும்:
1. `booking_flight_tickets` Beijing (PEK) முதல் Paris (CDG) வரை விவரங்களுடன்
2. `booking_hotels` Paris தங்குமிடம் விவரங்களுடன்

கீழே உள்ள செல்லை இயக்கி நேரடி உருவாக்கத்தைப் பார்க்கவும்:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## முடிவு

🎉 **வாழ்த்துக்கள்!** நீங்கள் ONNX Runtime GenAI-ஐ பயன்படுத்தி Phi-4 Mini உடன் சமமுறை function அழைப்புகளை வெற்றிகரமாக செயல்படுத்தியுள்ளீர்கள்.

### நீங்கள் என்ன கற்றுக்கொண்டீர்கள்:

1. **மாடல் அமைப்பு**: Phi-4 Mini-ஐ ONNX Runtime GenAI உடன் எப்படி அமைப்பது
2. **ஃபங்க்ஷன் வரையறுத்தல்**: AI function அழைப்புகளுக்கான function ஸ்கீமாக்களை எப்படி வரையறுப்பது
3. **வழிகாட்டப்பட்ட உருவாக்கம்**: கட்டமைக்கப்பட்ட வெளியீட்டு உருவாக்கத்திற்காக Lark grammar-ஐ எப்படி பயன்படுத்துவது
4. **சமமுறை function அழைப்புகள்**: பல function அழைப்புகள் தேவையான சிக்கலான கோரிக்கைகளை எப்படி கையாள்வது

### முக்கிய நன்மைகள்:

- ✅ **கட்டமைக்கப்பட்ட வெளியீடு**: வழிகாட்டப்பட்ட உருவாக்கம் செல்லுபடியாகும் JSON function அழைப்புகளை உறுதிப்படுத்துகிறது
- ✅ **சமமுறை செயலாக்கம்**: ஒரே கோரிக்கையில் பல function அழைப்புகளை கையாளவும்
- ✅ **உயர் செயல்திறன்**: ONNX Runtime மிகைப்படுத்தப்பட்ட கணிப்பை வழங்குகிறது
- ✅ **நெகிழ்வான ஸ்கீமா**: function வரையறைகளை சேர்க்க அல்லது மாற்ற எளிதாகும்

### வளங்கள்:

- [Phi-4 Mini Documentation](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Documentation](https://onnxruntime.ai/docs/genai/)
- [Function Calling Best Practices](https://platform.openai.com/docs/guides/function-calling)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
மறுப்பு அறிக்கை:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவையான Co-op Translator (https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சித்தாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கலாம் என்பதை கவனத்தில் கொள்ளவும். அசல் ஆவணம் அதன் மொழியில் (மூல மொழி) அதிகாரபூர்வமான ஆதாரமாகக் கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பை பரிந்துரைக்கிறோம். இந்த மொழிபெயர்ப்பின் பயன்பாட்டினால் ஏற்பட்ட எந்தவொரு தவறான புரிதலும் அல்லது தவறான பொருள் விளக்கத்திற்கும் நாங்கள் பொறுப்பல்லோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
